### Use ``Open Route Service API`` to retrieve distance matrix
#### Quota 500 per day, approx 1 year of merged rental-place dataset per day. 
#### Specify the ``year`` of rental and place dataset before running 

In [1]:
import pandas as pd
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../scripts/')
from add_distance import add_distance_time
import openrouteservice as ors

In [2]:
PLACE_COLS = ['Place_Names', 'latitude', 'longitude', 'year', 'place_type', 'SA2_CODE']
place_df = pd.read_csv('../../data/curated/features_of_interst/place_all_with_sa2.csv')[PLACE_COLS]
print(place_df.shape)

(70444, 6)


### Alter the place and property dataframes for specific year (2013, 2014, ... 2022)

In [3]:
place_df = place_df[place_df['year'] == 2014]
print(place_df.shape)
place_df.head(10)

(5929, 6)


,Place_Names,latitude,longitude,year,place_type,SA2_CODE
0,TAHARA BRIDGE RESERVE PICNIC AREA,-37.66568,141.66459,2013,park,217011420
1,CULGOA RECREATION RESERVE,-35.71682,143.10101,2013,park,215031400
2,CUMBERLAND,-38.57263,143.94681,2013,park,203031049
3,WAREENA PARK,-36.35911,146.30977,2013,park,204021066
4,ENTERPRIZE PARK,-37.82000,144.95973,2013,park,206041505
5,RYANS RESERVE,-37.66807,145.15741,2013,park,209031215
6,FERNTREE GULLY PICNIC GROUND,-37.89088,145.31692,2013,park,211051282
7,LORRAINE KOVACS RESERVE,-37.97763,145.25403,2013,park,212021454
8,RAY BASTIN RESERVE,-38.03359,145.30791,2013,park,212021456
9,NEVILLE HAMILTON RESERVE,-38.02612,145.34533,2013,park,212021293


In [4]:
PROPERTY_COLS = ['address', 'latitude', 'longitude', 'nbed', 'nbath', 'ncar', 'weekly_rent', 'type', 'postcode', 'year', 'month', 'residence_type', 'SA2_CODE']
property_df = pd.read_csv('../../data/curated/property_all_with_SA2/2014_property_with_SA2.csv')[PROPERTY_COLS]
# 2013 example
print(property_df.shape)
property_df.head(10)

(11466, 13)


,address,latitude,longitude,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE
0,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,300,House,3451,2013,11,House,202021027
1,"171 MAIN ROAD, CHEWTON",-37.081326,144.259245,3.0,1.0,8,330,House,3451,2013,9,House,202021027
2,"173 MAIN ROAD, CHEWTON",-37.080952,144.259378,2.0,1.0,0,270,House,3451,2013,10,House,202021027
3,"173 MAIN ROAD, CHEWTON",-37.080952,144.259378,2.0,1.0,0,270,House,3451,2013,9,House,202021027
4,"4 GEORGE STREET, CHEWTON",-37.082086,144.256513,3.0,1.0,2,350,House,3451,2013,4,House,202021027
5,"2/56 ALBERT ROAD, CHILTERN",-36.149080,146.604709,2.0,1.0,1,185,House,3683,2013,10,House,204031070
6,"8 SKERRY STREET, CHILTERN",-36.151158,146.598197,3.0,2.0,2,280,House,3683,2013,7,House,204031070
7,"4 GLENDALE DRIVE, CHIRNSIDE PARK",-37.751383,145.312411,4.0,2.0,2,350,Rental_residential,3116,2013,2,Other,211051275
8,"12-14 CAMPUS GATE, CHIRNSIDE PARK",-37.755523,145.332948,5.0,2.0,2,390,House,3116,2013,8,House,211051275
9,"12-14 CAMPUS GATE, CHIRNSIDE PARK",-37.755523,145.332948,5.0,2.0,2,390,House,3116,2013,6,House,211051275


In [5]:
merged_df = property_df.merge(place_df, how='inner', on='SA2_CODE')
print(merged_df.shape)
print(f"Distinct sa2 codes = {merged_df['SA2_CODE'].nunique()}")
merged_df = merged_df.rename(columns={'latitude_x': 'latitude_ori', 'longitude_x': 'longitude_ori', 'latitude_y': 'latitude_des', 'longitude_y': 'longitude_des', 'year_x': 'year'})
merged_df.drop('year_y', axis=1, inplace=True)
merged_df.head(15)

(153905, 18)
Distinct sa2 codes = 467


,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE,Place_Names,latitude_des,longitude_des,place_type
0,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,300,House,3451,2013,11,House,202021027,OLD DIAMOND HILL RESERVE,-37.08179,144.20126,park
1,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,300,House,3451,2013,11,House,202021027,CASTLEMAINE,-37.06316,144.21379,station
2,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,300,House,3451,2013,11,House,202021027,CHEWTON,-37.08485,144.25848,station
3,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,300,House,3451,2013,11,House,202021027,WINTERS FLAT PRIMARY SCHOOL,-37.07016,144.20749,primary
4,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,300,House,3451,2013,11,House,202021027,CASTLEMAINE NORTH PRIMARY SCHOOL,-37.05783,144.21899,primary
5,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,300,House,3451,2013,11,House,202021027,CASTLEMAINE PRIMARY SCHOOL,-37.06660,144.22233,primary
6,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,300,House,3451,2013,11,House,202021027,CHEWTON PRIMARY SCHOOL,-37.08233,144.26116,primary
7,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,300,House,3451,2013,11,House,202021027,CAMPBELLS CREEK PRIMARY SCHOOL,-37.09293,144.20466,primary
8,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,300,House,3451,2013,11,House,202021027,Castlemaine High School,-37.05054,144.22750,secondary
9,"208 MAIN ROAD, CHEWTON",-37.080111,144.261809,3.0,1.0,2,300,House,3451,2013,11,House,202021027,CASTLEMAINE TECHNICAL COLLEGE,-37.06346,144.22098,secondary


In [6]:
subset = merged_df[merged_df['SA2_CODE'] == 204021064].reset_index(drop=True)
count_places = len(subset.drop_duplicates(subset=['Place_Names','latitude_des']))
print(f"Subset size = {len(subset)}, Places count = {count_places}, Property count = {len(subset) // count_places}")
subset.head(60)

Subset size = 78, Places count = 6, Property count = 13


,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE,Place_Names,latitude_des,longitude_des,place_type
0,"30 PARK STREET, ABBOTSFORD",-37.808037,144.994189,3.0,2.0,1,670,none,3067,2013,7,Other,206071139,VICTORIA PARK,-37.79826,144.99657,park
1,"30 PARK STREET, ABBOTSFORD",-37.808037,144.994189,3.0,2.0,1,670,none,3067,2013,7,Other,206071139,VICTORIA PARK,-37.79916,144.99445,station
2,"30 PARK STREET, ABBOTSFORD",-37.808037,144.994189,3.0,2.0,1,670,none,3067,2013,7,Other,206071139,COLLINGWOOD,-37.80452,144.99375,station
3,"30 PARK STREET, ABBOTSFORD",-37.808037,144.994189,3.0,2.0,1,670,none,3067,2013,7,Other,206071139,VICTORIA PARK PRIMARY SCHOOL,-37.79735,144.99653,primary
4,"30 PARK STREET, ABBOTSFORD",-37.808037,144.994189,3.0,2.0,1,670,none,3067,2013,7,Other,206071139,ABBOTSFORD PRIMARY SCHOOL,-37.80898,144.99881,primary
5,"30 PARK STREET, ABBOTSFORD",-37.808037,144.994189,3.0,2.0,1,670,none,3067,2013,7,Other,206071139,VICTORIA PARK POST PRIMARY SCHOOL,-37.79735,144.99653,primary
6,"27 YARRA BANK COURT, ABBOTSFORD",-37.804738,144.998342,4.0,2.0,2,900,Townhouse,3067,2013,12,House,206071139,VICTORIA PARK,-37.79826,144.99657,park
7,"27 YARRA BANK COURT, ABBOTSFORD",-37.804738,144.998342,4.0,2.0,2,900,Townhouse,3067,2013,12,House,206071139,VICTORIA PARK,-37.79916,144.99445,station
8,"27 YARRA BANK COURT, ABBOTSFORD",-37.804738,144.998342,4.0,2.0,2,900,Townhouse,3067,2013,12,House,206071139,COLLINGWOOD,-37.80452,144.99375,station
9,"27 YARRA BANK COURT, ABBOTSFORD",-37.804738,144.998342,4.0,2.0,2,900,Townhouse,3067,2013,12,House,206071139,VICTORIA PARK PRIMARY SCHOOL,-37.79735,144.99653,primary


In [7]:
#df = pd.DataFrame({'firstname':['Philip', 'Philip', 'Andy', 'Jax', 'Jax'], 'surname': ['Cai', 'Tang', 'Xiao', 'Xiao', 'Zhuang']})
#df[['firstname', 'surname']].drop_duplicates()

#### Specify the ``year`` here

In [8]:
client = ors.Client('5b3ce3597851110001cf6248768e34108776435cabbcffaf2fb63a60') # kakakkaa api key 500

In [9]:
#added_distance_merged_df = add_distance_time(merged_df, 2013) # specify the year here
added_distance_merged_df = add_distance_time(subset, 2014, client, 'testing')

Executing SA2 Code 206071139
Subset size = 78, Places count = 6, Property count = 13
To Place Distance Grand List, Normal Branch, length = 78


In [10]:
added_distance_merged_df.head(60)

,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,...,longitude_des,place_type,dist_to_place_M,dist_to_place_KM,time_to_place_S,time_to_place_MIN,dist_to_cbd_M,dist_to_cbd_KM,time_to_cbd_S,time_to_cbd_MIN
0,"30 PARK STREET, ABBOTSFORD",-37.808037,144.994189,3.0,2.0,1,670,none,3067,2013,...,144.99657,park,2225.80,2.22580,253.11,4.218500,3076.43,3.07643,413.70,6.895000
1,"30 PARK STREET, ABBOTSFORD",-37.808037,144.994189,3.0,2.0,1,670,none,3067,2013,...,144.99445,station,2055.01,2.05501,305.93,5.098833,3076.43,3.07643,413.70,6.895000
2,"30 PARK STREET, ABBOTSFORD",-37.808037,144.994189,3.0,2.0,1,670,none,3067,2013,...,144.99375,station,1351.38,1.35138,189.07,3.151167,3076.43,3.07643,413.70,6.895000
3,"30 PARK STREET, ABBOTSFORD",-37.808037,144.994189,3.0,2.0,1,670,none,3067,2013,...,144.99653,primary,2241.27,2.24127,255.90,4.265000,3076.43,3.07643,413.70,6.895000
4,"30 PARK STREET, ABBOTSFORD",-37.808037,144.994189,3.0,2.0,1,670,none,3067,2013,...,144.99881,primary,760.93,0.76093,109.11,1.818500,3076.43,3.07643,413.70,6.895000
5,"30 PARK STREET, ABBOTSFORD",-37.808037,144.994189,3.0,2.0,1,670,none,3067,2013,...,144.99653,primary,2241.27,2.24127,255.90,4.265000,3076.43,3.07643,413.70,6.895000
6,"27 YARRA BANK COURT, ABBOTSFORD",-37.804738,144.998342,4.0,2.0,2,900,Townhouse,3067,2013,...,144.99657,park,1510.71,1.51071,174.97,2.916167,3817.56,3.81756,469.90,7.831667
7,"27 YARRA BANK COURT, ABBOTSFORD",-37.804738,144.998342,4.0,2.0,2,900,Townhouse,3067,2013,...,144.99445,station,1675.18,1.67518,242.28,4.038000,3817.56,3.81756,469.90,7.831667
8,"27 YARRA BANK COURT, ABBOTSFORD",-37.804738,144.998342,4.0,2.0,2,900,Townhouse,3067,2013,...,144.99375,station,636.29,0.63629,110.93,1.848833,3817.56,3.81756,469.90,7.831667
9,"27 YARRA BANK COURT, ABBOTSFORD",-37.804738,144.998342,4.0,2.0,2,900,Townhouse,3067,2013,...,144.99653,primary,1526.17,1.52617,177.76,2.962667,3817.56,3.81756,469.90,7.831667
